# Зависимости и т д

In [2]:
!git clone https://github.com/kechsin/TinyDialogues.git

Cloning into 'TinyDialogues'...
remote: Enumerating objects: 5760, done.
remote: Counting objects: 100% (976/976), done.
remote: Compressing objects: 100% (352/352), done.
remote: Total 5760 (delta 226), reused 922 (delta 192), pack-reused 4784 (from 3)
Receiving objects: 100% (5760/5760), 285.30 MiB | 17.08 MiB/s, done.
Resolving deltas: 100% (1301/1301), done.
Filtering content: 100% (4/4), 388.72 MiB | 26.49 MiB/s, done.


In [3]:
%cd TinyDialogues

/content/TinyDialogues


In [4]:
!pip uninstall -y transformers tokenizers
!pip install transformers==4.57.3 tokenizers==0.22.0

Found existing installation: transformers 4.57.3
Uninstalling transformers-4.57.3:
  Successfully uninstalled transformers-4.57.3
Found existing installation: tokenizers 0.22.1
Uninstalling tokenizers-0.22.1:
  Successfully uninstalled tokenizers-0.22.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 83.4 MB/s eta 0:00:00


In [5]:
!pip install -r transformers/examples/pytorch/language-modeling/requirements.txt
!pip install accelerate tokenizers nltk evaluate
!pip install numpy==1.24.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 84.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


# Обучаем GPT2 на TinyDialogues (синтетических данных)

## Загрузка датасета и токенизатор

In [14]:
import os
from datasets import load_dataset

In [15]:
ds = load_dataset("styfeng/TinyDialogues")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

tinydialogue_train_ordered.txt:   0%|          | 0.00/141M [00:00<?, ?B/s]

tinydialogue_val_ordered.txt:   0%|          | 0.00/25.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/110024 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/19708 [00:00<?, ? examples/s]

In [16]:
ds['train'][0]

{'text': '**Dad**: "Hey sweetie, do you want to paint with Daddy?" \\n\\n **Child**: "Paint!" \\n\\n **Dad**: "Yes, we\'ll use these brushes. But first, let\'s put on your apron so we don\'t get paint on your clothes." \\n\\n **Child**: "Apron!" \\n\\n **Mom**: "Breakfast is almost ready! Who wants pancakes?" \\n\\n **Child**: "Pancake!" \\n\\n **Dad**: "We\'ll eat first, then paint. Let\'s wash hands before we eat, okay?" \\n\\n **Child**: "Wash!" \\n\\n **Mom**: "Careful, the pancakes are hot. We\'ll let them cool a little bit." \\n\\n **Child**: "Hot?" \\n\\n **Mom**: "Yes, hot. But they\'ll be just right soon. Here\'s your plate." \\n\\n **Child**: "Eat pancake!" \\n\\n **Dad**: "After pancakes, we can start our painting. What colors do you want to use?" \\n\\n **Child**: "Blue!" \\n\\n **Mom**: "Blue is a great choice. Finish up and then you can paint with blue." \\n\\n **Child**: "Blue paint!" \\n\\n **Dad**: "Alright, pancakes are done. Let\'s clean up and then it\'s painting ti

In [17]:
output_folder = "TD"
os.makedirs(output_folder, exist_ok=True)

In [18]:
with open(os.path.join(output_folder, "train.txt"), "w", encoding="utf-8") as f:
    for text in ds["train"]:
        f.write(text["text"] + "<|endoftext|>" + "\n")

In [19]:
with open(os.path.join(output_folder, "val.txt"), "w", encoding="utf-8") as f:
    for text in ds["validation"]:
        f.write(text["text"] + "<|endoftext|>" + "\n")

In [20]:
!python scripts/tokenizers/train_GPT2_tokenizer.py TD/train.txt TD/val.txt my_TD_tokenizer

tokenizer_config.json: 100% 26.0/26.0 [00:00<00:00, 182kB/s]
config.json: 100% 665/665 [00:00<00:00, 4.11MB/s]
vocab.json: 100% 1.04M/1.04M [00:00<00:00, 4.85MB/s]
merges.txt: 100% 456k/456k [00:00<00:00, 2.03MB/s]
tokenizer.json: 100% 1.36M/1.36M [00:00<00:00, 3.25MB/s]
['def', 'Ġadd', '_', 'n', 'umbers', '(', 'a', ',', 'Ġb', '):', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġ"""', 'Add', 'Ġthe', 'Ġtwo', 'Ġnumbers', 'Ġ`', 'a', '`', 'Ġand', 'Ġ`', 'b', '`', '."', '""', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġreturn', 'Ġa', 'Ġ+', 'Ġb']
129732
[00:00:00] Tokenize words                 ██████████████████ 93421    /    93421
[00:00:00] Count pairs                    ██████████████████ 93421    /    93421
[00:00:01] Compute merges                 ██████████████████ 51742    /    51742
All special tokens: ['<|endoftext|>', '<UNK>']
BOS token: <|endoftext|>
EOS token: <|endoftext|>
PAD token: None
UNK token: <|endoftext|>
SEP token: None
CLS token: None
MASK token: None


In [21]:
!python scripts/tokenizers/test_GPT2_tokenizer.py my_TD_tokenizer



 my_TD_tokenizer 

['do', 'Ġyou', 'Ġwant', 'Ġto', 'Ġlook', 'Ġat', 'Ġthat', 'Ġit', 'Ġsays', 'Ġlook', 'Ġ', '?']
do you want to look at that it says look ? 

['The', 'Ġyellow', '-', 'billed', 'Ġshri', 'ke', 'Ġ(', "'", 'Cor', 'vin', 'ella', 'Ġcor', 'v', 'ina', "')", 'Ġis', 'Ġa', 'Ġlarge', 'Ġpasser', 'ine', 'Ġbird', 'Ġin', 'Ġthe', 'Ġshri', 'ke', 'Ġfamily', '.', 'ĠIt', 'Ġis', 'Ġsometimes', 'Ġknown', 'Ġas', 'Ġthe', 'Ġlong', '-', 'tailed', 'Ġshri', 'ke', ',', 'Ġbut', 'Ġthis', 'Ġis', 'Ġto', 'Ġbe', 'Ġdiscouraged', ',', 'Ġsince', 'Ġit', 'Ġinvites', 'Ġconfusion', 'Ġwith', 'Ġthe', 'Ġlong', '-', 'tailed', 'Ġshri', 'ke', ',', "Ġ'", 'L', 'an', 'ius', 'Ġsc', 'ha', 'ch', "',", 'Ġof', 'Ġtropical', 'Ġsouthern', 'ĠAsia', '.', 'ĠThe', 'Ġyellow', '-', 'billed', 'Ġshri', 'ke', 'Ġis', 'Ġa', 'Ġcommon', 'Ġresident', 'Ġbreeding', 'Ġbird', 'Ġin', 'Ġtropical', 'ĠAfrica', 'Ġfrom', 'ĠSen', 'egal', 'Ġeast', 'ward', 's', 'Ġto', 'ĠU', 'g', 'anda', 'Ġand', 'Ġlocally', 'Ġin', 'Ġwesternmost', 'ĠK', 'eny', 'a', '.', 'ĠIt'

Я посмотрела в коде этого теста на то, какие там предложения, и он их правильно написал, всё хорошо.
Можем ещё сравнить с токенайзером обученном ими (на тех же данных, и я ничего не меняла, так что должно быть то же самое)

In [22]:
!python scripts/tokenizers/test_GPT2_tokenizer.py tokenizers/GPT2_tinydialogue



 tokenizers/GPT2_tinydialogue 

['do', 'Ġyou', 'Ġwant', 'Ġto', 'Ġlook', 'Ġat', 'Ġthat', 'Ġit', 'Ġsays', 'Ġlook', 'Ġ', '?']
do you want to look at that it says look ? 

['The', 'Ġyellow', '-', 'billed', 'Ġshri', 'ke', 'Ġ(', "'", 'Cor', 'vin', 'ella', 'Ġcor', 'v', 'ina', "')", 'Ġis', 'Ġa', 'Ġlarge', 'Ġpasser', 'ine', 'Ġbird', 'Ġin', 'Ġthe', 'Ġshri', 'ke', 'Ġfamily', '.', 'ĠIt', 'Ġis', 'Ġsometimes', 'Ġknown', 'Ġas', 'Ġthe', 'Ġlong', '-', 'tailed', 'Ġshri', 'ke', ',', 'Ġbut', 'Ġthis', 'Ġis', 'Ġto', 'Ġbe', 'Ġdiscouraged', ',', 'Ġsince', 'Ġit', 'Ġinvites', 'Ġconfusion', 'Ġwith', 'Ġthe', 'Ġlong', '-', 'tailed', 'Ġshri', 'ke', ',', "Ġ'", 'L', 'an', 'ius', 'Ġsc', 'ha', 'ch', "',", 'Ġof', 'Ġtropical', 'Ġsouthern', 'ĠAsia', '.', 'ĠThe', 'Ġyellow', '-', 'billed', 'Ġshri', 'ke', 'Ġis', 'Ġa', 'Ġcommon', 'Ġresident', 'Ġbreeding', 'Ġbird', 'Ġin', 'Ġtropical', 'ĠAfrica', 'Ġfrom', 'ĠSen', 'eg', 'al', 'Ġeast', 'ward', 's', 'Ġto', 'ĠU', 'g', 'anda', 'Ġand', 'Ġlocally', 'Ġin', 'Ġwesternmost', 'ĠK', 'eny'

Да, вроде одинаково

## Обучаем модель

In [6]:
!bash scripts/language_model_training/GPT2_CHILDES_4-GPUs_train.sh --evaluation_strategy epoch\
  TD/train.txt TD/val.txt tokenizers/GPT2_tinydialogue \
  tokenizers/GPT2-small_config GPT2_TD_output \
  gpt2 1e-04 1 8 42

GPT2_TD_output
tokenizers/GPT2-small_config
gpt2
1e-04
1
2025-12-26 18:09:49.605022: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766772589.625801    2441 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766772589.631996    2441 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766772589.648829    2441 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766772589.648860    2441 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766772589.648864 